In [1]:
import torch
from datasets import load_dataset
import numpy as np


#定义数据集
class Dataset_train(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset("csv",data_dir="./", data_files="data_lite_big_train.csv", split=split)
        # self.dataset = load_dataset("csv",
        # data_dir="C:/Users/19148/Documents/Pycharm_projects/paper_project", data_files="data_lite.csv", split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label

class Dataset_test(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset("csv",data_dir="./", data_files="data_lite_big_test.csv", split=split)
        # self.dataset = load_dataset("csv",
        # data_dir="C:/Users/19148/Documents/Pycharm_projects/paper_project", data_files="data_lite.csv", split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']

        return text, label

dataset_train = Dataset_train('train')
print(len(dataset_train), dataset_train[15])
dataset_test = Dataset_test('train')
print(len(dataset_test), dataset_test[15])



Using custom data configuration default
Reusing dataset csv (C:\Users\19148\.cache\huggingface\datasets\csv\default-896d6d854b3b0933\0.0.0\2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)


2795 ('直接替换', 1)


Using custom data configuration default
Reusing dataset csv (C:\Users\19148\.cache\huggingface\datasets\csv\default-f4c04b1f5310396e\0.0.0\2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2)


869 ('求助', 3)


In [66]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=100,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset_train,
                                     batch_size=15,
                                     collate_fn=collate_fn,
                                     shuffle=False,
                                     # shuffle=True,
                                     drop_last=True)
len_loader = len(loader)
for i, (input_ids, attention_mask, token_type_ids,
        labels, input_tensor) in enumerate(loader):
    print("%d/%d"%(i+1,len_loader), end=' ')
    if i%8 == 7: print('\n', end='')
    # print(labels)
    # print(input_ids.shape)
    # break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

ValueError: not enough values to unpack (expected 5, got 4)

In [48]:
# data process
import jieba
import pandas as pd
from sklearn import preprocessing
import re
import numpy as np
from collections import deque
import operator

np.random.seed(0)


class DataPreprocess():
    def __init__(self,inputdata_path,stopword_path):
        self.dataed,self.labeled = self.dataload(inputdata_path,stopword_path)

    def dataload(self,inputdata_path,stopword_path):
        train_data = pd.read_csv(inputdata_path,delimiter='\t',
                                 lineterminator='\n',encoding='UTF-8')
        rawlabellist = []
        datalist = [[] for _ in range(train_data.shape[0])]
        for i in range(train_data.shape[0]):
            rawlabellist.append(train_data.iloc[i,0])
        labellist = preprocessing.LabelEncoder().fit_transform(rawlabellist)
        for i in range(train_data.shape[0]):
            datalist[i] = train_data.iloc[i,0]
        with open(stopword_path,'r',encoding="UTF-8") as f:
            self.stopword = []
            for line in f.readlines():
                self.stopword.append(line.strip())
        return datalist,labellist

    def stopworddelete(self,sentence):
        word_num = 0
        while(word_num<len(sentence)):
            if sentence[word_num] in self.stopword:
                sentence.remove(sentence[word_num])
                word_num -= 1
            word_num += 1
        return sentence

    def wordcut(self,sentence):
        criterion = r'[\u4e00-\u9fa5a-zA-Z]+' # 文本正则化，只留下汉字和英文
        sentence = re.findall(criterion,sentence)
        sentence = "".join(sentence) # 拼接列表为字符串
        sentence = jieba.lcut(sentence) # jieba分词
        sentence = self.stopworddelete(sentence) # 去停止词
        return sentence

    def savecutdata(self,datasave_path):
        with open(datasave_path,"w",encoding="UTF-8") as f:
            for i in range(len(self.dataed)):
                cut_words = self.wordcut(self.dataed[i])
                combin_word = ""
                for i in cut_words:
                    combin_word += i
                    combin_word += " "
                f.write(combin_word)
                f.write('\n')

class CutDataProcess:
    def __init__(self,cutdata_path,min_count):
        self.sentence_index = 0
        self.pos_pair = deque()
        self.min_count = min_count
        self.dataprocess(cutdata_path)

    def dataprocess(self,cutdata_path):
        cut_data = pd.read_csv(cutdata_path,header=None,lineterminator='\n',encoding='UTF-8',delimiter='\t')
        self.cut_list = []
        for i in range(cut_data.shape[0]):
            self.cut_list.append(cut_data.iloc[i][0].strip().split())
        word_frequency = dict()
        for i in self.cut_list: # 统计词频
            for j in i:
                try:
                    word_frequency[j] += 1
                except:
                    word_frequency[j] = 1
        self.word_id = dict()
        self.id_word = dict()
        self.word_frequency = []
        self.highword_frequency = dict()
        word_index = 0
        for word,count in word_frequency.items():
            if count < self.min_count:
                continue
            self.word_id[word] = word_index
            self.id_word[word_index] = word
            self.highword_frequency[word] = count
            word_index += 1
        self.word_count = len(self.word_id)
        self.sentence_count = len(self.cut_list)
        # 按词频重新排列字典,sorted输出为元组格式
        self.word_frequency = sorted(self.highword_frequency.items(),key=lambda x:x[1])
        self.allword_num = 0
        for item in self.highword_frequency.items():
            self.allword_num += item[1]

    def countpair(self,window_size):
        # 粗略估计正样本对的个数
        return self.allword_num*(2*window_size-1)-window_size*(
            window_size+1)*self.sentence_count

    def getpospair(self,batch_size,window_size):
        while len(self.pos_pair)<batch_size:
            try:
                sentence = self.cut_list[self.sentence_index]
            except: # iteration>1时要再次遍历数据
                self.sentence_index = 0
                sentence = self.cut_list[self.sentence_index]
            self.sentence_index += 1
            posword_id = []
            for word in sentence:
                try:
                    if word in self.highword_frequency:
                        posword_id.append(self.word_id[word])
                except:
                    continue
            for i,u in enumerate(posword_id): # i为中心词索引，u为中心词
                for j,v in enumerate(posword_id[max(i-window_size, 0):min(i+window_size+1,len(posword_id))]):
                    if i==j:
                        continue
                    self.pos_pair.append((u,v))

        posbatch_pair = []
        for _ in range(batch_size):
            posbatch_pair.append(self.pos_pair.popleft())
        return posbatch_pair

    def getnegv(self,posbatch_pair,neg_count):
        # 负采样频率设置为词频率的0.75次方,提高低频词的选中概率(只有数组元素可以在内部进行计算)
        sam_frequency = np.array(list(self.highword_frequency.values()))**0.75
        # 使用排序过后的词典选词(便于看清选词概率)
        # sam_frequency = np.array(list(dict(self.word_frequency).values()))**0.75
        # sam_probability是一个数组，里面是所有词预计被取为负样本的概率
        sam_probability = sam_frequency/sum(sam_frequency)
        maxpro_index,_ = max(enumerate(sam_probability),key=operator.itemgetter(1))
        if sum(sam_probability)>1:
            sam_probability[maxpro_index] = sum(sam_probability)-1+sam_probability[maxpro_index]
        else:
            sam_probability[maxpro_index] = 1-sum(sam_probability)+sam_probability[maxpro_index]
        # 按词频来挑选负样本
        neg_v = np.random.choice(range(len(sam_probability)),replace=True,size=(len(posbatch_pair),neg_count),p=sam_probability).tolist()
        return neg_v

# 处理原始数据，将数据分词、去停用词并保存为cutdata.txt为词嵌入做准备
if __name__ == '__main__':
    inputdata_path = r'.\data_lite_big_train.csv'
    stopword_path = r'.\cn_stopwords.txt'
    datasave_path = r'.\cut_train.txt'
    data = DataPreprocess(inputdata_path,stopword_path)
    data.savecutdata(datasave_path)
    inputdata_path = r'.\data_lite_big_test.csv'
    stopword_path = r'.\cn_stopwords.txt'
    datasave_path = r'.\cut_test.txt'
    data = DataPreprocess(inputdata_path,stopword_path)
    data.savecutdata(datasave_path)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\19148\AppData\Local\Temp\jieba.cache
Loading model cost 0.557 seconds.
Prefix dict has been built successfully.


In [50]:
# train word2vec get word embedding
from torch.autograd import Variable
import torch
from model import SkipGramModel
import torch.optim as optim
from tqdm import tqdm

class Word2Vec:
    def __init__(self,
                 input_file_name,
                 output_file_name,
                 emb_dimension=100,
                 batch_size=300,
                 window_size=5,
                 neg_count=5,
                 iteration=1,
                 initial_lr=0.005,
                 min_count=5):
        """
        Args:
            input_file_name: Name of the cutdata file. Each line is a sentence splited with space.
            output_file_name: Name of the final embedding file.
            emb_dimention: Embedding dimention, typically from 50 to 500.
            batch_size: The count of word pairs for one forward.
            window_size: Max skip length between words.
            neg_count: The num of negative samples correspond to a positive sample
            iteration: Controlling the number of training iterations.
            initial_lr: Initial learning rate.
            min_count: The minimal word frequency, words with lower frequency will be filtered.
        """
        self.data = CutDataProcess(input_file_name, min_count)
        self.output_file_name = output_file_name
        self.iteration = iteration
        self.emb_size = len(self.data.word_id)
        self.emb_dimension = emb_dimension
        self.neg_count = neg_count
        self.batch_size = batch_size
        self.window_size = window_size
        self.initial_lr = initial_lr
        self.skip_gram_model = SkipGramModel(self.emb_size, self.emb_dimension)
        self.device = "cpu"
        # self.device = ("cuda" if torch.cuda.is_available else "cpu")
        # if self.device=="cuda":
        #     self.skip_gram_model.to(self.device)
        self.optimizer = optim.SGD(
            self.skip_gram_model.parameters(), lr=self.initial_lr)

    def train(self):
        pair_count = self.data.countpair(self.window_size)
        batch_count = self.iteration*pair_count/self.batch_size
        process_bar = tqdm(range(int(batch_count)))
        for i in process_bar:
            pos_pairs = self.data.getpospair(self.batch_size,
                                                  self.window_size)
            neg_v = self.data.getnegv(pos_pairs,self.neg_count)
            pos_u = [pair[0] for pair in pos_pairs] # 取出中心词
            pos_v = [pair[1] for pair in pos_pairs] # 取出背景词

            pos_u = Variable(torch.LongTensor(pos_u)).to(self.device)
            pos_v = Variable(torch.LongTensor(pos_v)).to(self.device)
            neg_v = Variable(torch.LongTensor(neg_v)).to(self.device)

            self.optimizer.zero_grad()
            loss = self.skip_gram_model.forward(pos_u,pos_v,neg_v)
            loss.backward()
            self.optimizer.step()

            process_bar.set_description("Loss: %0.8f, lr: %0.6f" %
                                        (loss.item(),
                                         self.optimizer.param_groups[0]['lr']))
            if i*self.batch_size%100000==0:
                lr = self.initial_lr*(1-1*i/batch_count)
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] = lr
        self.skip_gram_model.savemodel(
            self.data.id_word, self.output_file_name)


if __name__ == '__main__':
    inputdata_path = r'.\data_lite_big_train.csv'
    stopword_path = r'.\cn_stopwords.txt'
    datasave_path = r'.\cut_train.txt'
    cutdata_path = datasave_path
    output_path = r'.\word_embedding_train.txt'
    # 数据预处理->分割成词
    # data = DataPreprocess(inputdata_path,stopword_path)
    # data.savecutdata(datasave_path)
    word2vec = Word2Vec(cutdata_path,output_path)
    word2vec.train()

    inputdata_path = r'.\data_lite_big_test.csv'
    stopword_path = r'.\cn_stopwords.txt'
    datasave_path = r'.\cut_test.txt'
    cutdata_path = datasave_path
    output_path = r'.\word_embedding_test.txt'
    # 数据预处理->分割成词
    # data = DataPreprocess(inputdata_path,stopword_path)
    # data.savecutdata(datasave_path)
    word2vec = Word2Vec(cutdata_path,output_path)
    word2vec.train()

Loss: 1247.60766602, lr: 0.005000: 100%|██████████| 63/63 [00:00<00:00, 243.24it/s]


In [44]:
f=pd.read_csv(r'.\word_embedding_train.txt')
print(f)
g=pd.read_csv(r'.\cut_train.txt',header=None)
print(g)

                                词语数量: 663    嵌入维度: 100
0    测试题 -0.006395771 0.0018183979 0.0048517864 0.0...
1    这个 -0.11625799 0.045478843 0.010680677 -0.0646...
2    难道 -0.0046932623 -0.0044586635 -0.003102618 -0...
3    不是 -0.050836984 0.022907417 0.011029675 -0.025...
4    算 -0.03897849 0.011776407 0.009073133 -0.02306...
..                                                 ...
658  以及 -0.009353659 0.008519782 -0.0027040928 -0.0...
659  日中值 -0.0051732264 0.0042957296 -0.0038952243 -...
660  令 0.00052627525 0.0009976791 0.0014530715 -0.0...
661  推导 -0.0046923077 0.0039344276 -0.004116141 -0....
662  底数 -0.008870682 0.00095511763 -0.0016141707 -0...

[663 rows x 1 columns]
                                                      0
0                           测试题 这个 难道 不是 转换成 xcosx 算 吗 
1                                次 根式 有 理化 这道题 做 对 了 吗 
2                                             不是 对 的 吗 
3     第二 题 疑问 老师 第二 题 为什么 极限 是 选 D 而 不是 C 不是 有 一个 概念...
4                           第五 题 怎么 

In [51]:
import pandas as pd
file_train = open('.\word_embedding_train.txt','r',encoding='utf-8')
word_train = open('.\cut_train.txt','r',encoding='utf-8')
dict_train = {}
list_train = []
matrix_train = np.zeros((2795,20,100))
for line in file_train.readlines():
    line = line.strip()
    k = line.split(' ')[0]
    v = line.split(' ')[1:]
    dict_train[k] = v
for line in word_train.readlines():
    line = line.strip()
    length = len(line.split(' '))
    temp = []
    for j in range(length):
        temp.append(line.split(' ')[j])
    list_train.append(temp)
print(list_train[0])
for i in range(len(list_train)):
    if len(list_train[i]) >= 20 : le=20 # 最大长度为20
    else: le=len(list_train[i])
    for j in range(le):
        matrix_train[i,j] = [float(i) for i in pd.array(dict_train.get(list_train[i][j],np.zeros(100)))]
print(matrix_train.shape)
print(list_train)
file_train.close()
word_train.close()

['测试题', '这个', '难道', '不是', '转换成', 'xcosx', '算', '吗']
(2795, 20, 100)
[['测试题', '这个', '难道', '不是', '转换成', 'xcosx', '算', '吗'], ['次', '根式', '有', '理化', '这道题', '做', '对', '了', '吗'], ['不是', '对', '的', '吗'], ['第二', '题', '疑问', '老师', '第二', '题', '为什么', '极限', '是', '选', 'D', '而', '不是', 'C', '不是', '有', '一个', '概念', '说', '常数', '的', '极限', '是', '它', '本身', '吗'], ['第五', '题', '怎么', '做', '呀', '详细', '点', '本人', '看得懂', '哈哈'], ['不', '知道', '对', '不', '对'], ['这个', '题', '怎么', '做'], ['你', '还有', '一个', '没', '替换'], ['难题', '不会'], ['一道', '证明题', '如图', '希望', '老师', '可以', '解答', '谢谢', '老师'], ['老师', '这个', '题', '怎么', '解', '呢', '麻烦', '您', '了'], ['请问', '老师', '为什么', '不能', '对', 'dxdy', '直接', '求导'], ['怎么', '做'], ['等价', '无穷小'], ['你们', '的', '习题', '都', '是', '学校', '里发', '的', '还是', '书', '上面', '的'], ['直接', '替换'], ['请教', '老师'], ['考察', '等价', '无穷小', '代换', '无穷小', '的', '性质', '结果', '为'], ['求教', '鸭', '怎么', '做'], ['老师', '可以', '再', '讲', '详细', '些', '吗'], ['为什么', '测试', '的', '题目', '没有', '解析'], ['求', '看看', '我', '这样', '做', '对', '吗', '我', '为啥', '求', '不', '出

In [45]:
f=pd.read_csv(r'.\word_embedding_test.txt')
print(f)
g=pd.read_csv(r'.\cut_test.txt',header=None)
print(g)

                                词语数量: 248    嵌入维度: 100
0    最后 -3.5141886e-05 -0.00085779396 0.0006391081 ...
1    那个 0.0009954256 -0.0038087857 0.0036111053 0.0...
2    问题 -0.0034211003 -0.004681038 -0.0006613766 -0...
3    答案 0.0020400262 -0.0010650493 -0.0024199232 0....
4    是 0.005841634 -0.0009405317 0.0029734569 -0.00...
..                                                 ...
243  几个 -0.0031855085 0.0036481426 0.0033927897 0.0...
244  变化 0.0015284963 0.00079092116 0.00027300444 -0...
245  a 0.002266146 0.0021075984 -0.0005427174 0.004...
246  嗯 -0.0029549412 0.004228098 -0.0028867882 -0.0...
247  换元 0.0033261587 -0.0032141488 0.0020323019 -0....

[248 rows x 1 columns]
                                                     0
0                                    最后 那个 问题 答案 是 什么 
1             第二个 和 第四个 咋 做 帮帮我 呀 第二个 和 第四个 咋 做 帮帮我 呀 
2                                              这样 行 吗 
3                                   如图望 老师 作答 谢谢 老师 了 
4                                        

In [53]:
file_test = open('.\word_embedding_test.txt', 'r', encoding='utf-8')
word_test = open('.\cut_test.txt', 'r', encoding='utf-8')
dict_test = {}
list_test = []
matrix_test = np.zeros((869, 20, 100))
for line in file_test.readlines():
    line = line.strip()
    k = line.split(' ')[0]
    v = line.split(' ')[1:]
    dict_test[k] = v
for line in word_test.readlines():
    line = line.strip()
    length = len(line.split(' '))
    temp = []
    for j in range(length):
        temp.append(line.split(' ')[j])
    list_test.append(temp)
for i in range(len(list_test)):
    if len(list_test[i]) >= 20:
        le = 20  # 最大长度为20
    else:
        le = len(list_test[i])
    for j in range(le):
        matrix_test[i, j] = [float(i) for i in pd.array(dict_test.get(list_test[i][j], np.zeros(100)))]
print(matrix_test.shape)
print(list_test)
file_test.close()
word_test.close()

(869, 20, 100)
[['最后', '那个', '问题', '答案', '是', '什么'], ['第二个', '和', '第四个', '咋', '做', '帮帮我', '呀', '第二个', '和', '第四个', '咋', '做', '帮帮我', '呀'], ['这样', '行', '吗'], ['如图望', '老师', '作答', '谢谢', '老师', '了'], ['太', '感谢', '了'], ['定', '积分', '老师', '我', '想', '问', '一下', '在', '到', '上', '的', '积分', 'fxdxC', '现在', '积分', '区间', '变为', '到', '结果', '是', 'C', '吗', '为什么'], ['还是', '不太', '明白', '怎么', '处理', '的', '能', '写', '一下', '步骤', '吗', '非常感谢'], ['这道题', '怎么', '做', '啊', '大神', '帮忙'], ['老师', '这道题', '请', '给出', '计算', '介导', '的', '过程', '我算', '的', '二阶', '导在', '处', '的', '值', '是', '是不是', '算错', '了'], ['哦哦哦', '我', '傻', '了'], ['题目', '不会'], ['这个', '怎么', '化'], ['的', '整数', '部分', '为什么', '是', '呢'], ['最终', '带', '着', 't'], ['怎么', '算', '的', '呀', '直接', 't', '我', '有点', '我太难', '了'], ['求助'], ['这是', '两个', '人', '的', '不同', '说法'], ['求', '大哥', '破解', '这道题'], ['那', '是', '阶乘'], ['你', '选', '的', '是', 'Asinx', '趋于', '无穷大', '时', '没有', '极限', '你', '可以', '取', '两个', '不同', '的', '收敛', '子例', '这', '与', '有', '极限', '的话', '极限', '唯一', '相', '矛盾', '了'], ['如何', '解此题', 'X

In [ ]:
class config(object):
    '''
    配置参数
    '''
    def __init__(self):
        # 类别数
        self.num_classes = 4

        # 整体训练次数
        self.num_epoch=3
        # batch大小
        self.batch_size=128
        #每个序列最大token数
        self.pad_size=32
        #学习率
        self.learning_rate = 1e-5

        self.bert_path='bert_pretrain'
        self.tokenizer=BertTokenizer.from_pretrained(self.bert_path) #定义分词器
        self.hidden_size=768  # Bert模型 token的embedding维度 = Bert模型后接自定义分类器（单隐层全连接网络）的输入维度

        # RNN 隐含层数量
        self.rnn_hidden_size=256
        # RNN数量
        self.num_layers=256
        # dropout
        self.dropout=0.5

In [79]:
from transformers import BertModel
from sklearn.preprocessing import MinMaxScaler
import torch
def standard_1(dim3_tensor):  # 归一化至-1 - 1
    get_min = dim3_tensor.min(dim=2)[0].unsqueeze(2) #取最小值张量 索引舍弃
    get_max = dim3_tensor.max(dim=2)[0].unsqueeze(2)
    normed_tensor = ((dim3_tensor - get_min) / (get_max - get_min) - 0.5) / 2
    return normed_tensor
# 定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 在每个RNN神经元之间使用 dropout 在单个神经元内部的两个时间步长间不使用 dropout
        self.lstm = torch.nn.LSTM(100,64,3,batch_first=True,bias=True,dropout=0.5)
        # self.lstm = torch.nn.LSTM(768,100,1,batch_first=True,bias=True, bidirectional=True)
        self.dropout = torch.nn.Dropout(0.5)
        # 双向LSTM要*2 分析LSTM节点数和网络层数时，看成神经元是LSTM全连接网络
        self.fc=torch.nn.Linear(64,4) # 自定义全连接层 ，输入数（输入的最后一个维度），输出数（多分类数量），bert模型输出的最后一个维度是768，这里的输入要和bert最后的输出统一


    def forward(self, input_tensor):
        # print(encoder)
        # encoder = standard_1(encoder)
        # print(encoder)
        # # bert output
        # out = self.fc(encoder[:, 0])
        # # out = out.last_hidden_state
        # out = out.softmax(dim=1)
        # print(encoder.shape, pooled.shape)
        output, _ = self.lstm(input_tensor)  # lstm层
        # output, _ = self.lstm(pooled)  # lstm层
        output =self.dropout(output)
        # print(output.shape)
        output =output[:,-1,:]   #encoder#只要序列中最后一个token对应的输出，（因为lstm会记录前边token的信息）
        output =self.fc(output)
        output = output.softmax(dim=1)

        return output

model = Model()
input_tensor = torch.from_numpy(matrix_train)
input_tensor = input_tensor[0:15]
input_tensor = input_tensor.to(torch.float32)
out = model(input_tensor)
print(out)
print(labels)
pre = out.argmax(dim=1)
print(pre)
out.shape

tensor([[0.2459, 0.2545, 0.2583, 0.2413],
        [0.2447, 0.2544, 0.2633, 0.2377],
        [0.2333, 0.2587, 0.2599, 0.2480],
        [0.2300, 0.2512, 0.2680, 0.2507],
        [0.2494, 0.2613, 0.2483, 0.2410],
        [0.2459, 0.2618, 0.2493, 0.2430],
        [0.2393, 0.2467, 0.2578, 0.2562],
        [0.2376, 0.2477, 0.2628, 0.2519],
        [0.2393, 0.2526, 0.2634, 0.2448],
        [0.2303, 0.2687, 0.2558, 0.2453],
        [0.2402, 0.2514, 0.2634, 0.2451],
        [0.2416, 0.2596, 0.2576, 0.2412],
        [0.2463, 0.2544, 0.2559, 0.2433],
        [0.2299, 0.2592, 0.2562, 0.2547],
        [0.2380, 0.2543, 0.2625, 0.2452]], grad_fn=<SoftmaxBackward0>)
tensor([2, 2, 2, 2, 3, 2, 0, 1, 2, 1, 0, 2, 2, 2, 0])
tensor([2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2])


torch.Size([15, 4])

In [63]:
from transformers import AdamW
loader_train = torch.utils.data.DataLoader(dataset=dataset_train,
                                     batch_size=13,
                                     collate_fn=collate_fn,
                                     shuffle=False,
                                     # shuffle=True,
                                     drop_last=True)
#训练
# optimizer = AdamW(model.parameters(), lr=5e-4)
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader_train):
    input_tensor = torch.from_numpy(matrix_train)
    input_tensor = input_tensor[13*i:13*i+13]
    input_tensor = input_tensor.to(torch.float32)
    out = model(input_tensor)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 1 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 300:
        break

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 1.3888522386550903 0.3076923076923077
1 1.3849151134490967 0.3076923076923077
2 1.3875807523727417 0.15384615384615385
3 1.3861438035964966 0.23076923076923078
4 1.3903121948242188 0.15384615384615385
5 1.3941553831100464 0.15384615384615385
6 1.385248064994812 0.3076923076923077
7 1.3852460384368896 0.07692307692307693
8 1.3825948238372803 0.46153846153846156
9 1.3805840015411377 0.3076923076923077
10 1.3925166130065918 0.23076923076923078
11 1.385306715965271 0.38461538461538464
12 1.3912746906280518 0.15384615384615385
13 1.3777607679367065 0.5384615384615384
14 1.3882758617401123 0.3076923076923077
15 1.3847229480743408 0.23076923076923078
16 1.393416166305542 0.23076923076923078
17 1.3905404806137085 0.0
18 1.3946564197540283 0.23076923076923078
19 1.3803348541259766 0.46153846153846156
20 1.3893038034439087 0.15384615384615385
21 1.3860058784484863 0.3076923076923077
22 1.3916720151901245 0.23076923076923078
23 1.3785605430603027 0.46153846153846156
24 1.3852566480636597 0.3846

In [64]:
#测试
from sklearn.metrics import classification_report
def test():
    model.eval()
    correct = 0
    total = 0
    label = torch.zeros(len(dataset_test))
    pre_label = torch.zeros(len(dataset_test))
    batch_size=11
    loader_test = torch.utils.data.DataLoader(dataset=dataset_test,
                                              batch_size=11,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        # if i == 5:
        #     break
        input_tensor = torch.from_numpy(matrix_test)
        input_tensor = input_tensor[11*i:11*i+11]
        input_tensor = input_tensor.to(torch.float32)

        print(i,end=' ')
        if (i+1)%10 == 0 :
            print('\n',end='')

        with torch.no_grad():
            out = model(input_tensor)

        out = out.argmax(dim=1)
        # print(out)
        for j in range(batch_size):  # batch_size=16
            label[batch_size * i + j] = labels[j]
            pre_label[batch_size * i + j] = out[j]
        correct += (out == labels).sum().item()
        total += len(labels)
    print('\n',end='')
    print(correct / total, correct, total)
    print(classification_report(label,pre_label,digits=4))


test()

0 1 2 3 4 5 6 7 8 9 
10 11 12 13 14 15 16 17 18 19 
20 21 22 23 24 25 26 27 28 29 
30 31 32 33 34 35 36 37 38 39 
40 41 42 43 44 45 46 47 48 49 
50 51 52 53 54 55 56 57 58 59 
60 61 62 63 64 65 66 67 68 69 
70 71 72 73 74 75 76 77 78 
0.3003452243958573 261 869
              precision    recall  f1-score   support

         0.0     0.0000    0.0000    0.0000       235
         1.0     0.0000    0.0000    0.0000       160
         2.0     0.3003    1.0000    0.4619       261
         3.0     0.0000    0.0000    0.0000       213

    accuracy                         0.3003       869
   macro avg     0.0751    0.2500    0.1155       869
weighted avg     0.0902    0.3003    0.1387       869



D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
